# 고급 상태 관리: 수정, 재개, Fork

이 노트북에서는 **상태 수정, 다양한 재개 방법, 히스토리에서 Fork**하는 고급 기능을 배웁니다.

## 고급 상태 관리 기능

```
┌────────────────────────────────────────────────────────────────────┐
│                    고급 상태 관리                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ get_state: 현재 상태 조회                                    │
│   2️⃣ update_state: 상태 수정                                      │
│   3️⃣ get_state_history: 과거 상태 히스토리 조회                   │
│   4️⃣ Fork: 과거 시점에서 분기하여 새로 실행                       │
│                                                                    │
│   활용:                                                            │
│   • 중단 시점에서 상태 수정 후 재개                                │
│   • 잘못된 결정 시 과거로 돌아가기                                 │
│   • What-if 시나리오 테스트                                        │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph duckduckgo-search langchain-community

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]

embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)

tools_retriever = InMemoryVectorStore.from_documents(
    [Document(tool.description, metadata={'name': tool.name}) for tool in tools],
    embeddings,
).as_retriever()


class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list[str]


def model_node(state: State) -> State:
    selected_tools = [tool for tool in tools if tool.name in state['selected_tools']]
    res = model.bind_tools(selected_tools).invoke(state['messages'])
    return {'messages': res}


def select_tools(state: State) -> State:
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {'selected_tools': [doc.metadata['name'] for doc in tool_docs]}


builder = StateGraph(State)
builder.add_node('select_tools', select_tools)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'select_tools')
builder.add_edge('select_tools', 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

graph = builder.compile(checkpointer=MemorySaver())

print("✅ 에이전트 그래프 준비 완료")

# 3. 상태 수정 후 재개 (update_state)

In [ ]:
input_data = {
    'messages': [
        HumanMessage(
            '미국 제30대 대통령의 사망 당시 나이는 몇 살이었나요?'
        )
    ]
}

config = {'configurable': {'thread_id': 'edit-state-1'}}

print("=== 1단계: 중단 ===")
output = graph.stream(input_data, config, interrupt_before=["model"])
for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
print("🛑 model 전에 중단됨\n")

In [ ]:
# 현재 상태 확인
state = graph.get_state(config)

print("=== 2단계: 현재 상태 확인 ===")
print(f"현재 질문: {state.values['messages'][0].content}")
print(f"선택된 도구: {state.values.get('selected_tools', [])}")

In [ ]:
# 상태 수정: 질문 변경
print("=== 3단계: 상태 수정 ===")

# 메시지 복사 후 첫 번째 메시지 변경
updated_messages = state.values['messages'].copy()
updated_messages[0] = HumanMessage(content="영화 보이후드의 촬영 기간은 얼마나 되나요?")

# 상태 업데이트 적용
update = {'messages': updated_messages}
graph.update_state(config, update)

# 수정된 상태 확인
new_state = graph.get_state(config)
print(f"변경된 질문: {new_state.values['messages'][0].content}")

In [ ]:
# 수정된 상태로 재개
print("=== 4단계: 수정된 상태로 재개 ===")

output = graph.stream(None, config)

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            if hasattr(last_msg, 'content') and last_msg.content:
                print(f"  응답: {str(last_msg.content)[:200]}...")
    print()

print("✅ 완료 (변경된 질문으로 실행됨)")

# 4. 히스토리 조회 (get_state_history)

In [ ]:
# 새 스레드로 전체 실행
config2 = {'configurable': {'thread_id': 'history-1'}}

print("=== 전체 실행 ===")
output = graph.stream(input_data, config2)
for chunk in output:
    for node_name in chunk.keys():
        print(f"노드 실행: {node_name}")

print("\n✅ 실행 완료")

In [ ]:
# 히스토리 조회
print("=== 상태 히스토리 ===")

history = list(graph.get_state_history(config2))

print(f"총 {len(history)}개의 상태 기록\n")

for i, state in enumerate(history):
    print(f"[{i}] 체크포인트: {state.config['configurable'].get('checkpoint_id', 'N/A')[:20]}...")
    print(f"    다음 노드: {state.next}")
    print(f"    메시지 수: {len(state.values.get('messages', []))}")
    print()

# 5. 히스토리에서 Fork

In [ ]:
# 과거 시점에서 다시 실행 (Fork)
print("=== 히스토리에서 Fork ===")

# 중간 시점의 상태 선택 (예: select_tools 완료 후)
if len(history) > 2:
    fork_state = history[2]  # 3번째 상태
    
    print(f"Fork 시점: {fork_state.next}")
    print(f"해당 시점의 config: {fork_state.config}")
    print()
    
    # 해당 시점에서 재실행
    print("=== Fork 지점에서 재실행 ===")
    output = graph.stream(None, fork_state.config)
    
    for chunk in output:
        for node_name in chunk.keys():
            print(f"노드 실행: {node_name}")
    
    print("\n✅ Fork 완료")
else:
    print("히스토리가 충분하지 않습니다.")

---

## 정리: 고급 상태 관리

### 핵심 API

```python
# 1. 현재 상태 조회
state = graph.get_state(config)
print(state.values)  # 상태 값
print(state.next)    # 다음 실행할 노드

# 2. 상태 수정
update = {'messages': new_messages}
graph.update_state(config, update)

# 3. 히스토리 조회
history = list(graph.get_state_history(config))
for state in history:
    print(state.config, state.next)

# 4. Fork (과거 시점에서 재실행)
output = graph.stream(None, history[2].config)
```

### 기능 비교

| 기능 | 용도 | API |
|------|------|-----|
| **get_state** | 현재 상태 확인 | `graph.get_state(config)` |
| **update_state** | 상태 수정 | `graph.update_state(config, update)` |
| **get_state_history** | 히스토리 조회 | `graph.get_state_history(config)` |
| **Fork** | 과거에서 분기 | `graph.stream(None, past_config)` |

### 사용 시나리오

| 시나리오 | 방법 |
|----------|------|
| 질문 수정 후 재실행 | update_state → stream(None) |
| 잘못된 결정 롤백 | get_state_history → Fork |
| What-if 테스트 | Fork로 여러 시나리오 실행 |
| 디버깅 | get_state_history로 각 단계 검토 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)
```

## ch08 요약: 고급 기능

| 기능 | 용도 | 핵심 API |
|------|------|----------|
| **Structured Output** | 구조화된 응답 | `with_structured_output()` |
| **Streaming** | 실시간 출력 | `stream()`, `astream_events()` |
| **Interrupt** | 중단/재개 | `interrupt_before`, `stream(None)` |
| **State Management** | 상태 수정/Fork | `get_state()`, `update_state()` |

## 다음 단계

**ch10에서는** RAG과 Agent의 **평가(Evaluation)** 방법을 배웁니다.